## 匯入相關套件

In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

import torch
import torch.nn as nn

import time
import math
import os
from datetime import datetime

import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")  

In [2]:
# 確認是否可以使用 GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
gpu_available = torch.cuda.is_available()

## 自定義函式、類別

In [3]:
class LSTM(nn.Module):

    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
      
        super(LSTM, self).__init__()
        
        self.hidden_dim = hidden_dim 
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(device).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(device).requires_grad_()
        
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :])
        
        return out

In [4]:
class GRU(nn.Module):
  
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        
        super(GRU, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(device).requires_grad_()
        
        out, (hn) = self.gru(x, (h0.detach()))
        out = self.fc(out[:, -1, :])
        
        return out

In [5]:
def stock(id, names):
    
    # 基本表格
    stockdate = f[f['name']==str(id)].loc[:,'date']
    stockdf = f[f['name']==str(id)].dropna(axis=1).loc[:,'open':'volume']
    
    oilnames = names[:10]
    ironnames = names[10:20]
    shipnames = names[20:]

    if id in oilnames:

      # 先剔除上面被選到的股票
      oilnames.remove(f'{id}')

      for remain in oilnames:

        remaindf = f[f['name']==remain].loc[:,'close']
        stockdf.insert(len(stockdf.columns), f'close_{remain}', list(remaindf))

    elif id in ironnames:

      # 先剔除上面被選到的股票
      ironnames.remove(f'{id}')

      for remain in ironnames:

        remaindf = f[f['name']==remain].loc[:,'close']
        stockdf.insert(len(stockdf.columns), f'close_{remain}', list(remaindf))


    elif id in shipnames:

      # 先剔除上面被選到的股票
      shipnames.remove(f'{id}')

      for remain in shipnames:
        
        remaindf = f[f['name']==remain].loc[:,'close']
        stockdf.insert(len(stockdf.columns), f'close_{remain}', list(remaindf))
    
    return stockdf, stockdate

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
def stockship(id):

    # 取出航運報價
    features = f[f['name']=='shippingIndex'].loc[:,['BCI','BCTI','BDI','BDTI','BPI','BSI']].reset_index(drop=True)

    stockdate = f[f['name']==str(id)].loc[:,'date']

    # 合併
    stockdf = f[f['name']==str(id)].dropna(axis=1).loc[:,['open','high','low','close','volume']].reset_index(drop=True)
    stockdf1 = pd.concat([stockdf, features], ignore_index=False, axis=1)
    
    return stockdf1, stockdate

In [8]:
def split_data(stock, lookback, gapspace=0, test_set_size=120):
    
    # data_raw = stock.to_numpy() # convert to numpy array
    data_raw = stock
    data = []
    
    # create all possible sequences of length seq_len
    for index in range(len(data_raw) - lookback): 

        data.append(data_raw[index: index + lookback,:])
    
    data = np.array(data);

    # test_set_size = int(np.round(0.2*data.shape[0]));
    train_set_size = data.shape[0] - (test_set_size);
    
    x_train = data[:train_set_size,:-1-gapspace,:]
    y_train = data[:train_set_size,-1,3:4]
    x_test = data[train_set_size:,:-1-gapspace,:]
    y_test = data[train_set_size:,-1,3:4]
  
    x_train = torch.from_numpy(x_train).type(torch.Tensor)
    y_train= torch.from_numpy(y_train).type(torch.Tensor)
    x_test = torch.from_numpy(x_test).type(torch.Tensor)
    y_test= torch.from_numpy(y_test).type(torch.Tensor)
            
    x_train = x_train.to(device)
    y_train= y_train.to(device)
    x_test = x_test.to(device)
    y_test= y_test.to(device)
        
    return x_train, y_train, x_test, y_test

In [9]:
def model_train_predict(ID=None, input_dim=None, epochs=500, model_name='LSTM',
             x_train=None, y_train=None, x_test=None, y_test=None, scalertar=None):
      
    mseloss_train = np.zeros(epochs)
    mseloss_test = np.zeros(epochs)
    all_y_train_pred = []
    all_y_test_pred = []
    
    if model_name == 'LSTM':
        
        model = LSTM(input_dim=input_dim, hidden_dim=32, output_dim=1, num_layers=2)
        model = model.to(device)
        
        criterion = torch.nn.MSELoss(reduction='mean')
        optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
     
    if model_name == 'GRU':
        
        model = GRU(input_dim=input_dim, hidden_dim=32, output_dim=1, num_layers=2)
        model = model.to(device)

        criterion = torch.nn.MSELoss(reduction='mean')
        optimiser = torch.optim.Adam(model.parameters(), lr=0.01)    

    for i in range(epochs):

        y_train_pred = model(x_train)
        loss = criterion(y_train_pred, y_train) 

        if (i+1)%25 == 0:

            print("Epoch ", i+1, "MSE: ", loss.item())

        mseloss_train[i] = loss.item()

        if gpu_available == False:  

            all_y_train_pred.append(scalertar.inverse_transform(y_train_pred.detach().numpy())) 

        else:

            all_y_train_pred.append(scalertar.inverse_transform(y_train_pred.cpu().detach().numpy()))                

        optimiser.zero_grad()
        loss.backward()
        optimiser.step()

        y_test_pred = model(x_test)
        losst = criterion(y_test_pred, y_test) 
        mseloss_test[i] = losst.item()

        if gpu_available == False:

            all_y_test_pred.append(scalertar.inverse_transform(y_test_pred.detach().numpy()))
        
        else:

            all_y_test_pred.append(scalertar.inverse_transform(y_test_pred.cpu().detach().numpy()))
            
    print('-'*80)        
    
    if os.path.exists('/content/drive/MyDrive/Final Project/_4_Build Models/_Final_Models/Model Evaluate Data/LSTM & GRU F5_Category/model') is False:

        os.makedirs('/content/drive/MyDrive/Final Project/_4_Build Models/_Final_Models/Model Evaluate Data/LSTM & GRU F5_Category/model')
    
    # 模型存檔
    #path = f'/content/drive/MyDrive/Final Project/_4_Build Models/_Final_Models/Model Evaluate Data/LSTM & GRU F5_Category/model/{ID}_{model_name}_epoch{epochs}.pth'
    #torch.save(model.state_dict(), path)
    
    return all_y_train_pred, all_y_test_pred, mseloss_train, mseloss_test


In [10]:
def Train_Data_Stock_Trend_Plot(y_train, y_train_pred, train_date_list=None, ID=None, model_name=None):
    
    fig = plt.figure(figsize=(16, 12))

    # 日期格式修正
    x= [datetime.strptime(str(d), '%Y-%m-%d').date() for d in train_date_list]

    ax = sns.lineplot(x=x, y=y_train.ravel(), label="Historical Data", color='royalblue')
    ax = sns.lineplot(x=x, y=y_train_pred.ravel(), label= f'{model_name} Train Prediction', color='tomato')
    ax.set_title(f'{ID}  {model_name} Train Data Stock Trend', size = 20, fontweight='bold')
    ax.set_xlabel('Year', size = 18)
    ax.set_ylabel('Stock Price (TWD)', size = 18)
    ax.set_xticklabels('', size=10)
    plt.legend(fontsize='xx-large', loc='upper left')

    #设置 x 轴主刻度格式
    alldays =  mdates.YearLocator()                           #主刻度为每年
    ax.xaxis.set_major_locator(alldays)                       #设置主刻度
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y')) 

    plt.legend(fontsize='xx-large', loc='upper left')
    plt.tick_params(labelsize = 16)
    #plt.gcf().autofmt_xdate()

    if os.path.exists('/content/drive/MyDrive/Final Project/_4_Build Models/_Final_Models/Model Evaluate Data/LSTM & GRU F5_Category/fig') is False:
      
        os.makedirs('/content/drive/MyDrive/Final Project/_4_Build Models/_Final_Models/Model Evaluate Data/LSTM & GRU F5_Category/fig')

    plt.show()
    #fig.savefig(f'/content/drive/MyDrive/Final Project/_4_Build Models/_Final_Models/Model Evaluate Data/LSTM & GRU F5_Category/fig/{ID}_{model_name}_Train_Data_Stock_Trend.png')

In [11]:
def Test_Data_Stock_Trend_Plot(y_test, y_test_pred, test_date_list=None, ID=None, model_name=None):
    
    fig = plt.figure(figsize=(16, 12))

    x= [datetime.strptime(str(d), '%Y-%m-%d').date() for d in test_date_list]

    ax = sns.lineplot(x=x, y=y_test.ravel(), label='Historical Data')
    ax = sns.lineplot(x=x, y=y_test_pred.ravel(), label=f'{model_name} Test Prediction')
    ax.set_title(f'{ID}  {model_name} Test Data Stock Trend', size = 20, fontweight='bold')
    ax.set_xlabel('Month', size = 18)
    ax.set_ylabel('Stock price (TWD)', size = 18)

    alldays =  mdates.MonthLocator()                          
    ax.xaxis.set_major_locator(alldays)                       
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m')) 

    plt.legend(fontsize='xx-large', loc='upper left')
    plt.tick_params(labelsize=16)
    #plt.gcf().autofmt_xdate()

    if os.path.exists('/content/drive/MyDrive/Final Project/_4_Build Models/_Final_Models/Model Evaluate Data/LSTM & GRU F5_Category/fig') is False:
      
        os.makedirs('/content/drive/MyDrive/Final Project/_4_Build Models/_Final_Models/Model Evaluate Data/LSTM & GRU F5_Category/fig')

    plt.show()
    #fig.savefig(f'/content/drive/MyDrive/Final Project/_4_Build Models/_Final_Models/Model Evaluate Data/LSTM & GRU F5_Category/fig/{ID}_{model_name}_Test_Data_Stock_Trend.png')

In [12]:
def LSTM_GRU_Data_Stock_Trend_Plot(y_test, lstmdata, grudata, test_date_list=None, ID=None):
    
    fig = plt.figure(figsize=(16, 12))

    x= [datetime.strptime(str(d), '%Y-%m-%d').date() for d in test_date_list]

    ax = sns.lineplot(x=x, y=y_test.ravel(), label='Historical Data')
    ax = sns.lineplot(x=x, y=lstmdata.ravel(), label=f'LSTM Test Prediction')
    ax = sns.lineplot(x=x, y=grudata.ravel(), label=f'GRU Test Prediction')
    ax.set_title(f'{ID}  Test Data Stock Trend', size = 20, fontweight='bold')
    ax.set_xlabel('Month', size = 18)
    ax.set_ylabel('Stock price (TWD)', size = 18)

    alldays =  mdates.MonthLocator()                          
    ax.xaxis.set_major_locator(alldays)                       
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m')) 

    plt.legend(fontsize='xx-large', loc='upper left')
    plt.tick_params(labelsize=16)
    #plt.gcf().autofmt_xdate()

    if os.path.exists('/content/drive/MyDrive/Final Project/_4_Build Models/_Final_Models/Model Evaluate Data/LSTM & GRU F5_Category/fig') is False:
      
        os.makedirs('/content/drive/MyDrive/Final Project/_4_Build Models/_Final_Models/Model Evaluate Data/LSTM & GRU F5_Category/fig')

    plt.show()
    #fig.savefig(f'/content/drive/MyDrive/Final Project/_4_Build Models/_Final_Models/Model Evaluate Data/LSTM & GRU F5_Category/fig/{ID}_Both_LSTM_GRU_Test_Data_Stock_Trend.png')

In [13]:
def RMSE_MSELOSS_Plot(allTrainRMSE, allTestRMSE, mseloss_train, mseloss_test, ID=None, model_name=None):
    
    fig = plt.figure()
    fig.subplots_adjust(hspace=0.2, wspace=0.2)

    plt.subplot(1, 2, 1)
    ax = sns.lineplot(data=allTrainRMSE, label='Train Data')
    ax = sns.lineplot(data=allTestRMSE, label='Test Data')
    ax.set_xlabel('Epoch', size = 16)
    ax.set_title(f'{ID}  {model_name} RMSE', size = 18, fontweight='bold')
    plt.tick_params(labelsize=14)
    plt.legend(fontsize='x-large', loc='upper right')

    plt.subplot(1, 2, 2)
    ax = sns.lineplot(data=mseloss_train, label='Train Data')
    ax = sns.lineplot(data=mseloss_test, label='Test Data')
    ax.set_xlabel('Epoch', size = 16)
    ax.set_title(f'{ID}  {model_name} MSE Loss', size = 18, fontweight='bold')
    plt.tick_params(labelsize=14)
    plt.legend(fontsize='x-large', loc='upper right')

    fig.set_figheight(6)
    fig.set_figwidth(16)
    
    if os.path.exists('/content/drive/MyDrive/Final Project/_4_Build Models/_Final_Models/Model Evaluate Data/LSTM & GRU F5_Category/fig') is False:
      
        os.makedirs('/content/drive/MyDrive/Final Project/_4_Build Models/_Final_Models/Model Evaluate Data/LSTM & GRU F5_Category/fig')

    plt.show()
    #fig.savefig(f'/content/drive/MyDrive/Final Project/_4_Build Models/_Final_Models/Model Evaluate Data/LSTM & GRU F5_Category/fig/{ID}_{model_name}_RMSE_MSELOSS.png')

In [14]:
def Stock_predict_process(ID, epochs, model_name, train_date_list=None, test_date_list=None):

    # 單獨 close 值的 scaler -> scalertar
    scalertar = MinMaxScaler()
    scalertar_data = scalertar.fit_transform(df.iloc[:,3].values.reshape(-1, 1))

    # 全部進行 scaler
    scaler = MinMaxScaler()
    scaler_data = scaler.fit_transform(df.values)

    # 分割資料，訓練維度(總天數, 以幾天來預測最後一天股價的天數，特徵值數量)，目標維度(總天數, 幾天後的股價值)
    x_train, y_train, x_test, y_test = split_data(scaler_data, lookback=lookback, gapspace=0, test_set_size=test_set_size)

    # 放入模型並訓練，得到預測值
    all_y_train_pred, all_y_test_pred, mseloss_train, mseloss_test = model_train_predict(ID=ID, input_dim=df.shape[1], 
                                                 epochs=epochs, model_name=model_name, 
                                                 x_train=x_train, y_train=y_train, 
                                                 x_test=x_test, y_test=y_test, 
                                                 scalertar=scalertar)

    # 將 tensor 換回 numpy
    if gpu_available == False:

        y_train = scalertar.inverse_transform(y_train.detach().numpy())
        y_test = scalertar.inverse_transform(y_test.detach().numpy())  

    else:

        y_train = scalertar.inverse_transform(y_train.cpu().detach().numpy())
        y_test = scalertar.inverse_transform(y_test.cpu().detach().numpy())

    # 算出每一個 epoch 中 訓練與測試的 RMSE值
    allTrainRMSE = []
    allTestRMSE = []

    for i in range(len(all_y_train_pred)):
      
        allTrainRMSE.append(math.sqrt(mean_squared_error(y_train[:,0], all_y_train_pred[i][:,0])))
        allTestRMSE.append(math.sqrt(mean_squared_error(y_test[:,0], all_y_test_pred[i][:,0])))


    # 畫圖
    # Train_Data_Stock_Trend_Plot
    Train_Data_Stock_Trend_Plot(y_train, all_y_train_pred[-1], train_date_list=train_date_list, ID=ID, model_name=model_name)
    
    # RMSE_MSELOSS_Plot
    RMSE_MSELOSS_Plot(allTrainRMSE, allTestRMSE, mseloss_train, mseloss_test, ID=ID, model_name=model_name)
    
    dfmseplot = pd.DataFrame({f'allTrainRMSE_{model_name}':allTrainRMSE,
                          f'allTestRMSE_{model_name}':allTestRMSE,
                          f'mseloss_train_{model_name}':mseloss_train,
                          f'mseloss_test_{model_name}':mseloss_test})
    
    # Test_Data_Stock_Trend_Plot    
    Test_Data_Stock_Trend_Plot(y_test, all_y_test_pred[-1], test_date_list=test_date_list, ID=ID, model_name=model_name)
    
    dfpred = pd.DataFrame({'date':train_date_list + test_date_list,
                       'split':['train']*len(train_date_list) + ['test']*len(test_date_list),
                       'historical':list(y_train.ravel()) + list(y_test.ravel()),
                       f'prediction_{model_name}':list(all_y_train_pred[-1].ravel()) + list(all_y_test_pred[-1].ravel())})
    
    
    # 計算 RMSE, R2, AdjR2
    trainRMSE = math.sqrt(mean_squared_error(y_train[:,0], all_y_train_pred[i][:,0]))
    testRMSE = math.sqrt(mean_squared_error(y_test[:,0], all_y_test_pred[i][:,0]))
    
    trainR2 = r2_score(y_train[:,0], all_y_train_pred[-1][:,0])
    testR2 = r2_score(y_test[:,0], all_y_test_pred[-1][:,0])

    trainAdjR2 = 1-(1-trainR2)*((len(x_train)-1)/(len(x_train)-x_train.shape[2]-1))
    testAdjR2 = 1-(1-testR2)*((len(x_test)-1)/(len(x_test)-x_test.shape[2]-1))

    dfscore = pd.DataFrame({'name':[f'{ID}', f'{ID}'],
                            'modelname':[f'{model_name}', f'{model_name}'],
                            'split': ['Train', 'Test'],
                            'RMSE': [trainRMSE, testRMSE],
                            'R2 Score': [trainR2, testR2],
                            'Adj R2 Score': [trainAdjR2, testAdjR2]})

    return dfmseplot, dfpred, dfscore

# START

In [15]:
# 檔案讀取大表
f = pd.read_csv('/content/drive/MyDrive/Final Project/_2_Acquire Clean Data/_Final_Dataset/_2_Clean NAN Data in Spark/clean_long_dataset.csv')

In [16]:
# 列出所有股票
names = list(f['name'].unique())[:30]

In [17]:
# 紀錄一下各自的時間
timelist = []
metrics_score = []

In [18]:
for name in names:
    
    # 參數設定
    ID = name 
    lookback = 10
    test_set_size = 120
    epochs = 500

    # 選取想要的股票
    df, alldate = stock(ID, names)
    # if name in names[:10]:
    #   df, alldate = stockoil(ID)
    # elif name in names[10:20]:
    #   df, alldate = stockiron(ID)
    # else:
    #   df, alldate = stockship(ID)

    # 保留日期資訊
    alldatelist = list(alldate)
    train_date_list = alldatelist[lookback:-test_set_size]
    test_date_list = alldatelist[-test_set_size:]

    # LSTM
    lstmstart = time.time()
    lstmdfmseplot, lstmdfpred, lstmdfscore = Stock_predict_process(ID, epochs, model_name = 'LSTM', 
                                     train_date_list=train_date_list, test_date_list=test_date_list)
    lstmend = time.time()
    timelist.append(lstmend - lstmstart)
    timelist.append(lstmend - lstmstart) # 為了後面整理資料

    # GRU
    grustart = time.time()
    grudfmseplot, grudfpred, grudfscore = Stock_predict_process(ID, epochs, model_name = 'GRU', 
                                    train_date_list=train_date_list, test_date_list=test_date_list)
    gruend = time.time()
    timelist.append(gruend - grustart)
    timelist.append(gruend - grustart)  # 為了後面整理資料

    # LSTM_GRU_Data_Stock_Trend_Plot
    LSTM_GRU_Data_Stock_Trend_Plot(lstmdfpred[lstmdfpred['split']=='test'].loc[:,'historical'].values,
                         lstmdfpred[lstmdfpred['split']=='test'].loc[:,'prediction_LSTM'].values, 
                         grudfpred[grudfpred['split']=='test'].loc[:,'prediction_GRU'].values,
                         test_date_list=test_date_list, ID=ID)

    # 存檔
    dfpred = pd.concat([lstmdfpred, grudfpred.loc[:,'prediction_GRU']], axis=1)
    dfmseplot = pd.concat([lstmdfmseplot, grudfmseplot], axis=1)
    dfscore = pd.concat([lstmdfscore, grudfscore]).reset_index(drop=True)

    if os.path.exists('/content/drive/MyDrive/Final Project/_4_Build Models/_Final_Models/Model Evaluate Data/LSTM & GRU F5_Category/researchdata') is False:
        os.makedirs('/content/drive/MyDrive/Final Project/_4_Build Models/_Final_Models/Model Evaluate Data/LSTM & GRU F5_Category/researchdata')

    #dfpred.to_csv(f'/content/drive/MyDrive/Final Project/_4_Build Models/_Final_Models/Model Evaluate Data/LSTM & GRU F5_Category/researchdata/{ID}_Prediction_Data.csv', index=False)
    #dfmseplot.to_csv(f'/content/drive/MyDrive/Final Project/_4_Build Models/_Final_Models/Model Evaluate Data/LSTM & GRU F5_Category/researchdata/{ID}_RMSE_MSELOSS_in_Training.csv', index=False)

    metrics_score.append(dfscore)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
all_metrics_score = pd.concat([x for x in metrics_score]).reset_index(drop=True)
all_metrics_score.insert(len(all_metrics_score.columns), 'time', timelist)
#all_metrics_score.to_csv(f'/content/drive/MyDrive/Final Project/_4_Build Models/_Final_Models/Model Evaluate Data/LSTM & GRU F5_Category/researchdata/All_Metrics_Score_Time.csv', index=False)

In [ ]:
lstm_time = []
gru_time = []

for i in range(len(timelist)):  

    if i%4 == 0:

        lstm_time.append(timelist[i])

    if i%4 == 2:

        gru_time.append(timelist[i])


In [ ]:
dfalltime = pd.DataFrame({'name':names[:30],
              'lstm_time':lstm_time,
              'gru_time':gru_time})

#dfalltime.to_csv(f'/content/drive/MyDrive/Final Project/_4_Build Models/_Final_Models/Model Evaluate Data/LSTM & GRU F5_Category/researchdata/All_Time.csv', index=False)